In [12]:
import pandas as pd
import os
import re
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))



# List files in the dataset directory and pick a supported file
supported_exts = [
    ".csv", ".tsv", ".json", ".jsonl", ".xml", ".parquet", ".feather",
    ".sqlite", ".sqlite3", ".db", ".db3", ".s3db", ".dl3",
    ".xls", ".xlsx", ".xlsm", ".xlsb", ".odf", ".ods", ".odt"
]
dataset_files = [f for f in os.listdir(path) if os.path.splitext(f)[1] in supported_exts]

if not dataset_files:
    raise ValueError("No supported dataset files found in the directory.")

file_path = os.path.join(path, dataset_files[0])

# Load the latest version
import pandas as pd

df = pd.read_csv(file_path)

print("First 5 records:", df.head())

[nltk_data] Downloading package stopwords to /Users/apple/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


First 5 records:                                           clean_text  category
0  when modi promised “minimum government maximum...      -1.0
1  talk all the nonsense and continue all the dra...       0.0
2  what did just say vote for modi  welcome bjp t...       1.0
3  asking his supporters prefix chowkidar their n...       1.0
4  answer who among these the most powerful world...       1.0


In [13]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Auto-load dataset
supported_exts = [".csv", ".tsv", ".json", ".jsonl"]
dataset_files = [f for f in os.listdir(path) if os.path.splitext(f)[1] in supported_exts]
if not dataset_files:
    raise ValueError("❌ No supported dataset files found in directory.")
file_path = os.path.join(path, dataset_files[0])

# Load
df = pd.read_csv(file_path)
print("📄 Shape:", df.shape)
df.head()


[nltk_data] Downloading package stopwords to /Users/apple/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


📄 Shape: (162980, 2)


,clean_text,category
0,when modi promised “minimum government maximum...,-1.0
1,talk all the nonsense and continue all the dra...,0.0
2,what did just say vote for modi welcome bjp t...,1.0
3,asking his supporters prefix chowkidar their n...,1.0
4,answer who among these the most powerful world...,1.0
